<a href="https://colab.research.google.com/github/maxreynolds/Pattern-Cloud/blob/main/StreamlitFinetuningApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup code
!cd /content/
!pip install -q streamlit
!pip install -q streamlit-image-select
!git clone https://github.com/huggingface/diffusers.git
!pip install -q ./diffusers
!pip install -q  -U -r /content/diffusers/examples/text_to_image/requirements.txt
!pip install -q wandb
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.7/706.7 kB 9.5 MB/s eta 0:00:00
Cloning into 'diffusers'...
remote: Enumerating objects: 42154, done.
remote: Counting objects: 100% (1952/1952), done.
remote: Compressing objects: 100% (818/818), done.
remote: Total 42154 (delta 1335), reused 1491 (delta 969), pack-reused 40202
Receiving objects: 100% (42154/42154), 27.81 MiB | 22.41 MiB/s, done.
Resolving deltas: 100% (31177/31177), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━

In [13]:
!accelerate config default --mixed_precision fp16

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://districts-hidden-bars-consolidated.trycloudflare.com


In [3]:
!streamlit run /content/home.py &>/content/logs.txt &

In [5]:
#Making directories for multipage Streamlit applicaiton files and storing user images
!mkdir /content/pages
!mkdir /content/user_images
!ls

cloudflared-linux-amd64  diffusers  logs.txt  nohup.out  pages	sample_data  user_images


In [6]:
%%writefile /content/home.py
import streamlit as st

import streamlit as st

st.set_page_config(
    page_title="Hello",
    page_icon="👋",
)

st.write("# Welcome to Streamlit! 👋")

st.sidebar.success("Select a demo above.")

st.markdown(
    """
    Streamlit is an open-source app framework built specifically for
    Machine Learning and Data Science projects.
    **👈 Select a demo from the sidebar** to see some examples
    of what Streamlit can do!
    ### Want to learn more?
    - Check out [streamlit.io](https://streamlit.io)
    - Jump into our [documentation](https://docs.streamlit.io)
    - Ask a question in our [community
        forums](https://discuss.streamlit.io)
    ### See more complex demos
    - Use a neural net to [analyze the Udacity Self-driving Car Image
        Dataset](https://github.com/streamlit/demo-self-driving)
    - Explore a [New York City rideshare dataset](https://github.com/streamlit/demo-uber-nyc-pickups)
"""
)




Writing /content/home.py


In [7]:
%%writefile /content/pages/1Upload.py
from diffusers import StableDiffusionPipeline
from datasets import load_dataset

import streamlit as st
import torch

from PIL import Image
import csv
import os


def delete_files_in_directory(directory_path):
   try:
     files = os.listdir(directory_path)
     for file in files:
       file_path = os.path.join(directory_path, file)
       if os.path.isfile(file_path):
         os.remove(file_path)
   except OSError:
     print("Error occurred while deleting files.")

image_set = []
image_set = st.file_uploader("Upload a folder of your training dataset Images:",
    type=["png"], accept_multiple_files=True) # TODO: expand to type=["png","jpg","jpeg"]]... need to update metadata.csv writer code to check for filetype

if image_set is not None:
    #st.write('Scroll down for next steps')
    st.image(image_set, width=128)

if st.button('Upload these Images as your training dataset', type="primary", use_container_width=True):
    placeholder = st.empty()
    placeholder.text("Formatting images...")
    for idx, image in enumerate(image_set):
      im = Image.open(image)
      im.save(f"/content/user_images/image{idx}.png")
    num_images = len(image_set)
    my_label = ""
    with open('/content/user_images/metadata.csv', 'w', newline='') as csvfile:
      spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
      spamwriter.writerow(['file_name,text'])
      for j in range(num_images):
        spamwriter.writerow([f'image{j}.png,'] + [my_label])
    placeholder.text('Loading dataset...')
    dataset = load_dataset("imagefolder", data_dir="/content/user_images", drop_labels=True)
    placeholder.text('Pushing to Hub...')
    dataset.push_to_hub("MaxReynolds/MyPatternDataset", private=False)
    placeholder.text('Upload complete!')
    #removing image files here
    delete_files_in_directory('/content/user_images')


Writing /content/pages/1Upload.py


In [8]:
%%writefile /content/pages/2Train.py
import streamlit as st
import subprocess

subprocess.run(["accelerate", "launch", "diffusers/examples/text_to_image/train_text_to_image.py",
  "--pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4",
  "--dataset_name=MaxReynolds/Lee_Souder_Combined",
  "--use_ema",
  "--resolution=512",
  "--center_crop",
  "--random_flip",
  "--train_batch_size=1",
  "--gradient_accumulation_steps=4",
  "--gradient_checkpointing",
  "--max_train_steps=400",
  "--learning_rate=1e-05",
  "--max_grad_norm=1",
  "--checkpointing_steps=100000",
  "--lr_scheduler=constant",
  "--lr_warmup_steps=0",
  "--push_to_hub",
  "--output_dir=MaxReynolds/MyStreamlitModel",
  "--validation_prompt=Rocket Launcher by Lee Souder",
  "--report_to=wandb"])


Writing /content/pages/2Train.py


In [9]:
%%writefile /content/pages/3Generate.py
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

import streamlit as st
from streamlit_image_select import image_select

model_path = "MaxReynolds/MyStreamlitApp"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

# Run inference using ChatGPT prompts to acquire 4 image panels
image1 = pipe(prompt="Rocket Lancher by Lee Souder").images[0]
image1.save("panel_1.png")

image2 = pipe(prompt="Rocket Launcher by Lee Souder").images[0]
image2.save("panel_2.png")

image3 = pipe(prompt="Rocket Launcher by Lee Souder").images[0]
image3.save("panel_3.png")

image4 = pipe(prompt="Rocket Launcher by Lee Souder").images[0]
image4.save("panel_4.png")

all_images = [image1, image2, image3, image4]

img = image_select(
    label="Select an image",
    images= all_images,
    return_value="index",
    key="image_select",
)

Writing /content/pages/3Generate.py
